# Negative control: comparing identically generated, unperturbed cohorts

## imports + setup

In [1]:
import logging
import warnings

import cloudpathlib
import helpers
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
warnings.filterwarnings("ignore", message="divide by zero")
warnings.filterwarnings("ignore", message="invalid value encountered in double_scalars")

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")

# helpers.logging.configure_logging()
# udp_handler = logging.handlers.DatagramHandler("localhost", 12000)
# udp_handler.setLevel("DEBUG")
# logging.getLogger().addHandler(udp_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(helpers.logging.formatter)
stream_handler.setLevel("INFO")
logging.getLogger().addHandler(stream_handler)

In [4]:
logging.getLogger("helpers").setLevel("DEBUG")

In [5]:
path_root = cloudpathlib.CloudPath(
    "gs://liulab/evaluating_cibersortx/identically_generated_cohorts_100/2022-09-14_19:37:57"
)

In [6]:
!gsutil ls -hlR $path_root



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

INFO 0914 21:04:21.790380 retry_util.py] Retrying request, attempt #4...
INFO 0914 21:04:35.184843 retry_util.py] Retrying request, attempt #5...
INFO 0914 21:04:59.635505 retry_util.py] Retrying request, attempt #6...
INFO 0914 21:05:31.858270 retry_util.py] Retrying request, attempt #7...
INFO 0914 21:06:03.923799 retry_util.py] Retrying request, attempt #8...
INFO 0914 21:06:35.973645 retry_util.py] Retrying request, attempt #9...
INFO 0914 21:07:08.113526 retry_util.py] Retrying request, attempt #10...
^C
Caught CTRL-C (signal 2) - exiting


In [7]:
# janky plotly hack
px.line(x=[1, 2], y=[1, 2])

## load data and results

## DEG analysis

### Comparing gene expression in simulated bulk RNA-seq

In [9]:
import importlib
importlib.reload(helpers.deg_analysis)

<module 'helpers.deg_analysis' from '/home/jupyter/deconv/helpers/deg_analysis.py'>

In [8]:
df_bulk_rnaseq = helpers.data_io_and_formatting.load_concatenated_bulk_rnaseq(
    path_root / "data" / "bulkrnaseq.txt")

cohort_id                                no_perturbations_1  \
GeneSymbol sample_id                                          
A1BG       TCGA-3N-A9WB-06A-11R-A38C-07          113.839703   
           TCGA-3N-A9WC-06A-11R-A38C-07           37.796220   
           TCGA-BF-AAP0-06A-11R-A39D-07           39.781144   
           TCGA-D3-A1Q3-06A-11R-A18T-07           63.064430   
           TCGA-D3-A1Q5-06A-11R-A18T-07           16.496005   
...                                                     ...   
ZZZ3       TCGA-GN-A4U4-06A-11R-A32P-07            0.782295   
           TCGA-HR-A2OG-06A-21R-A18U-07            5.491985   
           TCGA-WE-A8ZX-06A-11R-A37K-07            9.234191   
           TCGA-YG-AA3P-06A-11R-A38C-07           73.656539   
           TCGA-Z2-AA3S-06A-11R-A39D-07           65.233609   

cohort_id                                no_perturbations_2  
GeneSymbol sample_id                                         
A1BG       TCGA-3N-A9WB-06A-11R-A38C-07           16.548933  
           TCGA-3N-A9WC-06A-11R-A38C-07           24.874189  
           TCGA-BF-AAP0-06A-11R-A39D-07           18.760724  
           TCGA-D3-A1Q3-06A-11R-A18T-07           17.427817  
           TCGA-D3-A1Q5-06A-11R-A18T-07          102.289767  
...                                                     ...  
ZZZ3       TCGA-GN-A4U4-06A-11R-A32P-07           36.108743  
           TCGA-HR-A2OG-06A-21R-A18U-07           47.471332  
           TCGA-WE-A8ZX-06A-11R-A37K-07           35.222933  
           TCGA-YG-AA3P-06A-11R-A38C-07          162.861163  
           TCGA-Z2-AA3S-06A-11R-A39D-07           24.421787  

[803150 rows x 2 columns]

In [10]:
groups = df_bulk_rnaseq.groupby("GeneSymbol")
df_bulk_rnaseq_stats = helpers.deg_analysis.compute_stats(groups)
df_bulk_rnaseq_stats

,GeneSymbol,pval,fold_change,sparsity_overall,-log10_pval,log2_fold_change,-log10_pval_signed,significant_bh_fdr=0.5,pval_adj_bh,-log10_pval_adj_bh,-log10_pval_adj_bh_signed
0,A1BG,0.390738,1.180582,0.01,0.408115,0.239499,0.408115,False,1.0,-0.0,-0.0
1,A2M,0.239836,1.212499,0.00,0.620085,0.277983,0.620085,False,1.0,-0.0,-0.0
2,A2ML1,0.958765,0.851313,0.00,0.018288,-0.232238,-0.018288,False,1.0,-0.0,0.0
3,A4GALT,0.497107,0.959941,0.03,0.303551,-0.058982,-0.303551,False,1.0,-0.0,0.0
4,A4GNT,0.257732,23.467142,0.91,0.588832,4.552570,0.588832,False,1.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...
16058,ZYG11A,0.475530,0.930321,0.00,0.322822,-0.104200,-0.322822,False,1.0,-0.0,0.0
16059,ZYG11B,0.764268,0.970503,0.00,0.116754,-0.043196,-0.116754,False,1.0,-0.0,0.0
16060,ZYX,0.641693,1.040088,0.00,0.192673,0.056706,0.192673,False,1.0,-0.0,-0.0
16061,ZZEF1,0.849638,1.019877,0.00,0.070766,0.028396,0.070766,False,1.0,-0.0,-0.0


In [11]:
fig = make_volcano_figure(df_bulk_rnaseq_stats)
fig.update_layout(title="Differential expression in independent, identically-generated in silico cohorts")
fig

### Comparing gene expression in malignant cells, inferred by CIBERSORTx

In [ ]:
df_rnaseq_malignant_inferred = helpers.running_cibersortx.

data_io_and_formatting.load_concatenated_bulk_rnaseq(
    path_root / "data" / "bulkrnaseq.txt")